In [ ]:
from langchain_core.tools import tool

@tool
def verify_transaction(transaction_ids: list) -> list:
    """Verify each transaction ID for accuracy."""
    # Simulating verification process
    verified_transactions = [tx_id for tx_id in transaction_ids if tx_id.is_valid()]
    return verified_transactions

In [1]:
from langgraph.graph import StateGraph

# Define the research team graph
research_team_graph = StateGraph()
research_team_graph.add_node("DataCollector", data_collector_node)
research_team_graph.add_node("TransactionVerifier", transaction_verifier_node)

# Define edges to route tasks between agents and back to the team supervisor
research_team_graph.add_edge("DataCollector", "TransactionVerifier")
research_team_graph.add_edge("TransactionVerifier", "ResearchTeamSupervisor")

ValueError: Must provide state_schema or input and output

In [ ]:
@tool
def perform_financial_analysis(data: dict) -> dict:
    """Calculate financial metrics from the provided data."""
    results = {
        "debt_to_equity_ratio": data["total_debt"] / data["total_equity"],
        "current_ratio": data["current_assets"] / data["current_liabilities"],
    }
    return results

In [ ]:
from pathlib import  Path

@tool
def write_report(content: str, file_name: str) -> str:
    """Write the audit report to a text file."""
    report_path = Path(f"/reports/{file_name}")
    report_path.write_text(content)
    return f"Report saved at {report_path}"

In [ ]:
from langchain_experimental.utilities import PythonREPL
import matplotlib.pyplot as plt

repl = PythonREPL()

@tool
def generate_chart(data:dict, chart_type: str) -> str:
    """Generate a chart for financial data."""
    code = f""" 
import matplotlib.pyplot as plt
data = {data}
plt.figure(figsize=(10, 6))
if "{chart_type}" == "bar":
    plt.bar(data.keys(), data.values())
if "{chart_type}" == "line":
    plt.plot(data.keys(), data.values())
plt.title("Financial Metrics")
plt.savefig('/charts/financial_chart.png')
"""
    repl.run(code)
    return "Chart generated and saved at /charts/financial_chart.png"

In [4]:
# Define the Document Authoring Team Graph
document_authoring_graph = StateGraph()
document_authoring_graph.add_node("FinancialAnalyst", financial_analyst_node)
document_authoring_graph.add_node("ReportWriter", report_writer_node)
document_authoring_graph.add_node("ChartGenerator", chart_generator_node)

# Define edges to route tasks within the document authoring team
document_authoring_graph.add_edge("FinancialAnalyst", "ReportWriter")
document_authoring_graph.add_edge("ReportWriter", "ChartGenerator")
document_authoring_graph.add_edge("ChartGenerator", "DocumentAuthoringSupervisor")

ValueError: Must provide state_schema or input and output

In [ ]:
from langgraph.graph import StateGraph, START, END
# Define the Overall Supervisor Node
overall_supervisor = create_team_supervisor(
    llm, 
    "Manage financial audit workflow between data collection and report generation team.",
    ["ResearchTeam", "DocumentAuthoringTeam"]
)

# Define the top-level graph for the Financial Audit Team
financial_audit_graph = StateGraph()
financial_audit_graph.add_node("ResearchTeam", research_team_graph)
financial_audit_graph.add_node("DocumentAuthoringTeam", document_authoring_graph)
financial_audit_graph.add_node("OverallSupervisor", overall_supervisor)

# Define routing through the Overall Supervisor
financial_audit_graph.add_conditional_edges(
    "OverallSupervisor",
    lambda x: x["next"],
    {
        "ResearchTeam": "ResearchTeam",
        "DocumentAuthoringTeam": "DocumentAuthoringTeam",
        "FINISH": END
    }
)
financial_audit_graph.add_edge(START, "OverallSupervisor")